In [ ]:
#import dependencies
import pandas as pd

#link to resource files
school_data_csv = '../Resources/schools_complete.csv'
student_data_csv = '../Resources/students_complete.csv'

#read csv files
school_data = pd.read_csv(school_data_csv)
student_data = pd.read_csv(student_data_csv)
#merge csv files
complete_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

complete_data

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [ ]:
#district summary
#total schools
tot_schools = len(complete_data["School ID"].unique())

#total students
tot_students = len(complete_data['Student ID'])

#total budget
tot_budget = sum(complete_data['budget'].unique())

#average math score
ave_math = complete_data['math_score'].mean()

#average reading score
ave_read = complete_data['reading_score'].mean()


#percentage of students who passed reading  
passreading = complete_data.loc[complete_data['reading_score']>=70].count()
per_read_pass = passreading[1]/tot_students

#percentage of students who passed math
passmath = complete_data.loc[complete_data['math_score']>=70].count()
per_math_pass = passmath[1]/tot_students

#percentage of students who passed math AND reading
passboth = complete_data.loc[(complete_data['reading_score']>=70) & (complete_data['math_score']>=70)].count()
per_both_pass = passboth[1]/tot_students


district_summary = {'Total Schools':tot_schools, 
                    'Total Students': tot_students,
                    'Total Budget': tot_budget,
                    'Average Math Score': ave_math,
                    'Average Reading Score': ave_read,
                    '% Passing Math': per_math_pass,
                    '% Passing Reading':per_read_pass,
                    '% Overall Passing': per_both_pass}

district_df = pd.DataFrame([district_summary])
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,0.749809,0.858055,0.651723


In [ ]:
#school summary

#create new dataframe with necessary columns from complete_data
school_df = complete_data[['type', 'size', 'budget']]
#set the school name as the index
school_df = school_df.set_index(complete_data['school_name'])

#calculate and add column for the per student budget
stu_bud = school_df['budget']/school_df['size']
school_df['Per Student Budget'] = stu_bud

#calculate average math and reading scores
school_df['Total Math Score'] =(complete_data.groupby(school_df.index)['math_score'].sum())
school_df['Average Math Score'] = school_df['Total Math Score']/school_df['size']

school_df['Total Reading Score'] =(complete_data.groupby(school_df.index)['reading_score'].sum())
school_df['Average Reading Score'] = school_df['Total Reading Score']/school_df['size']

#calculate percent pass for math
math_pass_df = complete_data[['school_name','math_score']]
math_pass_df['pass math'] = complete_data['math_score'] >= 70
school_df['math pass count'] = math_pass_df.groupby(school_df.index)['pass math'].sum()
school_df['% Passing Math'] = school_df['math pass count']/school_df['size']
#Calculate percet pass for reading
read_pass_df = complete_data[['school_name','reading_score']]
read_pass_df['pass reading'] = complete_data['reading_score'] >= 70
school_df['reading pass count'] = read_pass_df.groupby(school_df.index)['pass reading'].sum()
school_df['% Passing Reading'] = school_df['reading pass count']/school_df['size']
#Calculate percet pass for both
both_pass_df = complete_data[['school_name','math_score', 'reading_score']]
both_pass_df['both pass'] = (complete_data['math_score'] >= 70) & (complete_data['reading_score'] >= 70)
school_df['both pass count'] = both_pass_df.groupby(school_df.index)['both pass'].sum()
school_df['% Overall Passing'] = school_df['both pass count']/school_df['size']

#eliminate extra columns
school_df = school_df[['type', 'size', 'budget', 'Per Student Budget', 
                       'Average Math Score', 'Average Reading Score', 
                       '% Passing Math', '% Passing Reading', '% Overall Passing']]

#delete duplicate school rows
school_df = school_df.drop_duplicates(keep='first')
#make it alphabetical
school_df = school_df.sort_values('school_name')
#change column names
school_df = school_df.rename(columns={'type': 'School Type', 
                                      'size': 'Total Students', 
                                      'budget': 'Total School Budget'})
school_df

/var/folders/gg/tmfkv_8j7v74s1jdyjk90y1c0000gn/T/ipykernel_25007/4124396857.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  math_pass_df['pass math'] = complete_data['math_score'] >= 70
/var/folders/gg/tmfkv_8j7v74s1jdyjk90y1c0000gn/T/ipykernel_25007/4124396857.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read_pass_df['pass reading'] = complete_data['reading_score'] >= 70
/var/folders/gg/tmfkv_8j7v74s1jdyjk90y1c0000gn/T/ipykernel_25007/4124396857.py:31: SettingWithCopyWarning: 
A value is tryin

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.546423
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.542899
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,0.925059,0.962529,0.892272
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.535392


In [ ]:
#Top Performing Schools (By % Overall Passing)
top_df = school_df.sort_values('% Overall Passing', ascending=False)
top_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,0.932722,0.973089,0.909480
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.905405


In [ ]:
#Bottom Performing Schools (By % Overall Passing)
bottom_df = school_df.sort_values('% Overall Passing')
bottom_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,0.663666,0.802201,0.529882
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.535392


In [ ]:
#Math Scores By Grade
math_grade_df = complete_data[['grade', 'math_score', 'size']]
#set the school name as the index
math_grade_df = math_grade_df.set_index(complete_data['school_name'])


#new DF for math grades for each grade to display math grades per grade per school
nine_df = math_grade_df.loc[math_grade_df['grade'] == '9th']
nine_df['total'] = nine_df.groupby(nine_df.index)['math_score'].sum()
nine_df['9perschool'] = nine_df.groupby(nine_df.index)['size'].count()
nine_df['nine_ave'] = nine_df['total']/nine_df['9perschool']
nine_df = nine_df.drop_duplicates(subset = 'size', keep='first')
nine_df = nine_df[['nine_ave']]

ten_df = math_grade_df.loc[math_grade_df['grade'] == '10th']
ten_df['total'] = ten_df.groupby(ten_df.index)['math_score'].sum()
ten_df['10perschool'] = ten_df.groupby(ten_df.index)['size'].count()
ten_df['ten_ave'] = ten_df['total']/ten_df['10perschool']
ten_df = ten_df.drop_duplicates(subset = 'size', keep='first')
ten_df = ten_df[['ten_ave']]

eleven_df = math_grade_df.loc[math_grade_df['grade'] == '11th']
eleven_df['total'] = eleven_df.groupby(eleven_df.index)['math_score'].sum()
eleven_df['11perschool'] = eleven_df.groupby(eleven_df.index)['size'].count()
eleven_df['eleven_ave'] = eleven_df['total']/eleven_df['11perschool']
eleven_df = eleven_df.drop_duplicates(subset = 'size', keep='first')
eleven_df = eleven_df[['eleven_ave']]

twelve_df = math_grade_df.loc[math_grade_df['grade'] == '12th']
twelve_df['total'] = twelve_df.groupby(twelve_df.index)['math_score'].sum()
twelve_df['12perschool'] = twelve_df.groupby(twelve_df.index)['size'].count()
twelve_df['twelve_ave'] = twelve_df['total']/twelve_df['12perschool']
twelve_df = twelve_df.drop_duplicates(subset = 'size', keep='first')
twelve_df = twelve_df[['twelve_ave']]

#merge grade level data frames
math_1 = pd.merge(nine_df, ten_df, how="left", on=["school_name"])
math_2 = pd.merge(eleven_df, twelve_df, how ="left", on=['school_name'])
math_final = pd.merge(math_1, math_2, how="left", on=['school_name'])

#remane columns

math_final = math_final.rename(columns={'nine_ave': '9th', 
                                        'ten_ave': '10th',
                                      'eleven_ave': '11th', 
                                      'twelve_ave': '12th'})
#sort alphabetically
math_final = math_final.sort_values('school_name')
math_final


/var/folders/gg/tmfkv_8j7v74s1jdyjk90y1c0000gn/T/ipykernel_25007/913945171.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nine_df['total'] = nine_df.groupby(nine_df.index)['math_score'].sum()
/var/folders/gg/tmfkv_8j7v74s1jdyjk90y1c0000gn/T/ipykernel_25007/913945171.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nine_df['9perschool'] = nine_df.groupby(nine_df.index)['size'].count()
/var/folders/gg/tmfkv_8j7v74s1jdyjk90y1c0000gn/T/ipykernel_25007/913945171.py:11: SettingWithCopyWarning: 
A value i

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [ ]:
#Reading Scores By Grade
read_grade_df = complete_data[['grade', 'reading_score', 'size']]
#set the school name as the index
read_grade_df = read_grade_df.set_index(complete_data['school_name'])


#new DF for math grades for each grade to display math grades per grade per school
nine_df = read_grade_df.loc[read_grade_df['grade'] == '9th']
nine_df['total'] = nine_df.groupby(nine_df.index)['reading_score'].sum()
nine_df['9perschool'] = nine_df.groupby(nine_df.index)['size'].count()
nine_df['nine_ave'] = nine_df['total']/nine_df['9perschool']
nine_df = nine_df.drop_duplicates(subset = 'size', keep='first')
nine_df = nine_df[['nine_ave']]

ten_df = read_grade_df.loc[read_grade_df['grade'] == '10th']
ten_df['total'] = ten_df.groupby(ten_df.index)['reading_score'].sum()
ten_df['10perschool'] = ten_df.groupby(ten_df.index)['size'].count()
ten_df['ten_ave'] = ten_df['total']/ten_df['10perschool']
ten_df = ten_df.drop_duplicates(subset = 'size', keep='first')
ten_df = ten_df[['ten_ave']]

eleven_df = read_grade_df.loc[read_grade_df['grade'] == '11th']
eleven_df['total'] = eleven_df.groupby(eleven_df.index)['reading_score'].sum()
eleven_df['11perschool'] = eleven_df.groupby(eleven_df.index)['size'].count()
eleven_df['eleven_ave'] = eleven_df['total']/eleven_df['11perschool']
eleven_df = eleven_df.drop_duplicates(subset = 'size', keep='first')
eleven_df = eleven_df[['eleven_ave']]

twelve_df = read_grade_df.loc[read_grade_df['grade'] == '12th']
twelve_df['total'] = twelve_df.groupby(twelve_df.index)['reading_score'].sum()
twelve_df['12perschool'] = twelve_df.groupby(twelve_df.index)['size'].count()
twelve_df['twelve_ave'] = twelve_df['total']/twelve_df['12perschool']
twelve_df = twelve_df.drop_duplicates(subset = 'size', keep='first')
twelve_df = twelve_df[['twelve_ave']]

#merge grade level data frames
read_1 = pd.merge(nine_df, ten_df, how="left", on=["school_name"])
read_2 = pd.merge(eleven_df, twelve_df, how ="left", on=['school_name'])
read_final = pd.merge(read_1, read_2, how="left", on=['school_name'])

#remane columns

read_final = read_final.rename(columns={'nine_ave': '9th', 
                                        'ten_ave': '10th',
                                      'eleven_ave': '11th', 
                                      'twelve_ave': '12th'})
#sort alphabetically
read_final = read_final.sort_values('school_name')
read_final

/var/folders/gg/tmfkv_8j7v74s1jdyjk90y1c0000gn/T/ipykernel_25007/2214130009.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nine_df['total'] = nine_df.groupby(nine_df.index)['reading_score'].sum()
/var/folders/gg/tmfkv_8j7v74s1jdyjk90y1c0000gn/T/ipykernel_25007/2214130009.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nine_df['9perschool'] = nine_df.groupby(nine_df.index)['size'].count()
/var/folders/gg/tmfkv_8j7v74s1jdyjk90y1c0000gn/T/ipykernel_25007/2214130009.py:11: SettingWithCopyWarning: 
A v

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [ ]:
#Scores by School Spending

#create new data frame with information from school_df
spend_df = school_df
spend_df = spend_df[['Per Student Budget', 'Average Math Score', 'Average Reading Score', 
                       '% Passing Math', '% Passing Reading', '% Overall Passing']]


#set index to per student budget
spend_df = spend_df.set_index(spend_df['Per Student Budget'])

#bin into 4 groups
bins = [0,585, 630, 645, 680]
labels = ['<$585','$585-630','$630-645','$645-680']
spend_df['Spending Ranges (Per Student)']=pd.cut(spend_df['Per Student Budget'], bins, 
                                                 labels = labels, include_lowest = True)


#group by school spending
spend_range = spend_df.groupby('Spending Ranges (Per Student)')
#average student scores per spending bin
spend_range.count()
spend_range[['Average Math Score','Average Reading Score',
           '% Passing Math','% Passing Reading','% Overall Passing']].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,0.934601,0.966109,0.903695
$585-630,81.899826,83.155286,0.871335,0.927182,0.814186
$630-645,78.518855,81.624473,0.734842,0.843918,0.628577
$645-680,76.997210,81.027843,0.661648,0.811340,0.535269


In [ ]:
#Scores by School Size

#create new data frame with information from school_df
size_df = school_df
size_df = school_df[['Total Students', 'Average Math Score', 'Average Reading Score', 
                       '% Passing Math', '% Passing Reading', '% Overall Passing']]

#set index to school size
size_df = size_df.set_index(size_df['Total Students'])

#bin into 3 groups
bins = [0,1000, 2000, 5000]
labels = ['Small (<1000)','Medium (1000-2000)','Large (2000-5000)']
size_df['School Size']=pd.cut(size_df['Total Students'], bins, labels = labels, include_lowest = True)

#group by school size
size_range = size_df.groupby('School Size')
#average student scores per spending bin
size_range.count()
size_range[['Average Math Score','Average Reading Score',
           '% Passing Math','% Passing Reading','% Overall Passing']].mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,0.935502,0.960994,0.898839
Medium (1000-2000),83.374684,83.864438,0.935997,0.967907,0.906215
Large (2000-5000),77.746417,81.344493,0.699634,0.827666,0.582860


In [ ]:
#Scores by School Type

#create new data frame with information from school_df
type_df = school_df
type_df = school_df[['School Type', 'Average Math Score', 'Average Reading Score', 
                       '% Passing Math', '% Passing Reading', '% Overall Passing']]

#group by school size
gb_type = type_df.groupby('School Type')
#average student scores per spending bin
gb_type.count()
gb_type[['Average Math Score','Average Reading Score',
           '% Passing Math','% Passing Reading','% Overall Passing']].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,0.936208,0.965865,0.904322
District,76.956733,80.966636,0.665485,0.807991,0.536722
